<a href="https://colab.research.google.com/github/glitchheadgit/smilesChessFraud/blob/main/chessFraud_Egor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and imports

In [39]:
!pip install transformers==4.18.0 datasets -q

In [11]:
!pip install pyarrow==15.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.0 which is incompatible.


In [20]:
import pandas as pd
import numpy as np

import torch
from tqdm.auto import tqdm, trange

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizerFast
from transformers import DataCollatorWithPadding
from tokenizers import BertWordPieceTokenizer
from torch.utils.data import DataLoader
from datasets import *
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# Datasets

In [7]:
ds = pd.read_parquet("part-00000.parquet")



# Train tokinizer

Make corpus of moves

In [8]:
moves = []
out_file = open("Moves.txt","w")
for i in ds["Moves_clean"]:
  for j in i:
    moves.append(j)
    print(j,file = out_file,end = ' ')


['Nf3', 'Nf6', 'b3', 'b6', 'Bb2', 'Bb7', 'd4', 'g6', 'e3', 'Bg7', 'Be2', 'O-O', 'O-O', 'e6', 'c4', 'c5', 'dxc5', 'bxc5', 'Qc2', 'Qe7', 'Nc3', 'Na6', 'Rac1', 'd5', 'cxd5', 'exd5', 'Rfd1', 'Rfd8', 'Na4', 'Rac8', 'Bxa6', 'Bxa6', 'Nxc5', 'Ne4', 'Bxg7', 'Rxc5', 'Qb2', 'Rdc8', 'Bd4', 'Rxc1', 'Rxc1', 'Qd8', 'Ne5', 'f6', 'Nc6', 'Qd7', 'Nxa7', 'e4', 'c5', 'Nf3', 'Nc6', 'd4', 'cxd4', 'Nxd4', 'Nf6', 'Nc3', 'e6', 'a3', 'Qc7', 'f4', 'd6', 'Be3', 'a6', 'Qf3', 'Be7', 'Be2', 'O-O', 'O-O-O', 'Rb8', 'g4', 'Nd7', 'g5', 'b5', 'Nf5', 'exf5', 'Nd5', 'Qd8', 'exf5', 'Re8', 'f6', 'Bxf6', 'gxf6', 'Nxf6', 'Nxf6+', 'Qxf6', 'Qxc6', 'Bb7', 'Qc7', 'Bxh1', 'Rxh1', 'Qe7', 'Qxe7', 'Rxe7', 'Kd2', 'Rbe8', 'Bxb5', 'axb5', 'Bd4', 'Re4', 'Kd3', 'Rxf4', 'b3', 'g5', 'a4', 'bxa4', 'bxa4', 'Ra8', 'Ra1', 'Ra5', 'Bb6', 'd4', 'Nf6', 'c4', 'g6', 'Bg5', 'Bg7', 'Nc3', 'h6', 'Bd2', 'd6', 'Nf3', 'Nc6', 'd5', 'Nb8', 'h3', 'e5', 'e4', 'a5', 'Be3', 'Na6', 'g4', 'Nd7', 'a3', 'Ndc5', 'b4', 'Nd7', 'Rg1', 'O-O', 'Rg3', 'Kh7', 'g5', 'h5', 'c5'

In [45]:
ds = ds.assign(Moves_clean=ds["Moves_clean"].apply(lambda l: " ".join(l)))

In [9]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]

files = ["Moves.txt"]

vocab_size = 30_522

max_length = 300
# whether to truncate
truncate_longer_samples = False

# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

Save tokenizer

In [10]:
import os
import json
model_path = "pretrained-bert"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)
tokenizer.save_model(model_path)



['pretrained-bert/vocab.txt']

In [11]:
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [12]:
Tokenizer = BertTokenizerFast.from_pretrained(model_path)


# Encoding


In [53]:
arr = []
for i in ds["Moves_clean"]:
  arr.append(tokenizer.encode(i).tokens)
ds["Moves_emb"] = arr
dataset = Dataset.from_pandas(ds,split = "train")
d = dataset.train_test_split(test_size=0.1)

'''def encode(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer.encode(examples["Moves_clean"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)
'''
train_dataset = d["train"]
# tokenizing the testing dataset
test_dataset = d["test"]

In [54]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

Grouping texts in chunks of 300:   0%|          | 0/5022 [00:00<?, ? examples/s]

TypeError: 'int' object is not iterable

In [ ]:
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)